In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
from pathlib import Path

import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd
import seaborn as sns

In [3]:
src_path: str = "../src"
sys.path.append(src_path)

In [4]:
ROOT: Path = Path("/media/ssd/Perez/storage/PCTA_WCDT_GSE221601_FILTERED")
FIGURES_PATH: Path = ROOT.joinpath("manuscript_figures")
FIGURES_PATH.mkdir(parents=True, exist_ok=True)

## Figure 6/7: WGCNA Network

---


### A: prim/norm

#### Up-regulated modules

In [ ]:
hub_genes_df = pd.read_csv(
    ROOT.joinpath("wgcna")
    .joinpath("sample_type_hspc+mcrpc+norm+prim__prim_vs_norm_padj_0_05_up_1_0")
    .joinpath("standard")
    .joinpath("results")
    .joinpath("bicor_signed_hub_genes.csv"),
    index_col=0,
)
mod_hub_genes = hub_genes_df["SYMBOL"].to_dict()
del mod_hub_genes["M0"]
mod_hub_genes

In [6]:
network_file = (
    ROOT.joinpath("rich_network")
    .joinpath("sample_type_hspc+mcrpc+norm+prim__prim_vs_norm_padj_0_05_up_1_0")
    .joinpath("standard")
    .joinpath("random_forest")
    .joinpath(
        "sample_type_hspc+mcrpc+norm+prim__"
        "prim_vs_norm_padj_0_05_up_1_0_corr_th_0_1_full_wgcna_ml_net.graphml"
    )
)
graph = nx.read_graphml(network_file)

In [ ]:
graph_nodes_df = pd.DataFrame(dict(graph.nodes)).transpose().sort_values("module")
graph_nodes_df["shap_value"] = graph_nodes_df["shap_value"].astype(float)
graph_nodes_df["log2FoldChange"] = graph_nodes_df["log2FoldChange"].astype(float)
graph_nodes_df.head()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 10), dpi=300)

ax = sns.stripplot(
    data=graph_nodes_df,
    x="module",
    y="shap_value",
    hue="module",
    dodge=False,
    jitter=False,
    ax=ax,
)

ax = sns.boxplot(
    data=graph_nodes_df,
    x="module",
    y="shap_value",
    hue="module",
    dodge=False,
    ax=ax,
)

# annotate top genes by SHAP value
for mod_id, (mod, mod_df) in enumerate(graph_nodes_df.groupby("module")):
    mod_df.sort_values("shap_value", ascending=False, inplace=True)
    text_y = 99
    for i in range(min(5, len(mod_df))):
        gene_meta = mod_df.iloc[i]
        if gene_meta["shap_value"] > 0.2:
            text_y_new = gene_meta["shap_value"] + 0.03
            text_y_new = (
                (text_y := text_y_new)
                if text_y - text_y_new > 0.03
                else (text_y := text_y - 0.03)
            )
            plt.annotate(
                gene_meta.name,
                xy=(mod_id, gene_meta["shap_value"]),
                xytext=(mod_id + 0.2, text_y_new),
                arrowprops=dict(facecolor="black", arrowstyle="->"),
                fontsize=10,
            )

# annotate module hub genes
for mod_id, (mod, hub_gene) in enumerate(mod_hub_genes.items()):
    plt.annotate(
        hub_gene,
        xy=(mod_id, graph_nodes_df.loc[hub_gene, "shap_value"]),
        xytext=(mod_id - 0.1, -0.05),
        ha="left",
        arrowprops=dict(facecolor="black", arrowstyle="->", connectionstyle="Angle3"),
        fontweight="bold",
        fontsize=12,
    )

plt.ylabel("SHAP value", fontsize=14)
plt.xlabel("WGCNA Module", fontsize=14)
plt.xticks(
    np.arange(0, graph_nodes_df["module"].nunique(), 1),
    [f"{m}\n(n={c})" for m, c in graph_nodes_df["module"].value_counts().items()],
    fontsize=12,
)
plt.yticks(fontsize=12)
plt.title(
    "Distribution of shapely values\nWGCNA prim/norm (up-regulated DEGs)", fontsize=16
)
ax.set_axisbelow(True)
plt.grid(axis="y", zorder=-10)
plt.ylim((-0.1, 0.6))
plt.xlim((-1, graph_nodes_df["module"].nunique()))
plt.legend([], [], frameon=False)
plt.tight_layout()
plt.savefig(FIGURES_PATH.joinpath("wgcna_prim_norm_up.pdf"))
plt.savefig(FIGURES_PATH.joinpath("wgcna_prim_norm_up.svg"))

#### Down-regulated modules

In [ ]:
hub_genes_df = pd.read_csv(
    ROOT.joinpath("wgcna")
    .joinpath("sample_type_hspc+mcrpc+norm+prim__prim_vs_norm_padj_0_05_down_1_0")
    .joinpath("standard")
    .joinpath("results")
    .joinpath("bicor_signed_hub_genes.csv"),
    index_col=0,
)
mod_hub_genes = hub_genes_df["SYMBOL"].to_dict()
del mod_hub_genes["M0"]
mod_hub_genes

In [10]:
network_file = (
    ROOT.joinpath("rich_network")
    .joinpath("sample_type_hspc+mcrpc+norm+prim__prim_vs_norm_padj_0_05_down_1_0")
    .joinpath("standard")
    .joinpath("random_forest")
    .joinpath(
        "sample_type_hspc+mcrpc+norm+prim__"
        "prim_vs_norm_padj_0_05_down_1_0_corr_th_0_1_full_wgcna_ml_net.graphml"
    )
)
graph = nx.read_graphml(network_file)

In [ ]:
graph_nodes_df = pd.DataFrame(dict(graph.nodes)).transpose().sort_values("module")
graph_nodes_df["shap_value"] = graph_nodes_df["shap_value"].astype(float)
graph_nodes_df["log2FoldChange"] = graph_nodes_df["log2FoldChange"].astype(float)
graph_nodes_df.head()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 10), dpi=300)

ax = sns.stripplot(
    data=graph_nodes_df,
    x="module",
    y="shap_value",
    hue="module",
    dodge=False,
    jitter=False,
    ax=ax,
)

ax = sns.boxplot(
    data=graph_nodes_df,
    x="module",
    y="shap_value",
    hue="module",
    dodge=False,
    ax=ax,
)

# annotate top genes by SHAP value
for mod_id, (mod, mod_df) in enumerate(graph_nodes_df.groupby("module")):
    mod_df.sort_values("shap_value", ascending=False, inplace=True)
    text_y = 99
    for i in range(min(5, len(mod_df))):
        gene_meta = mod_df.iloc[i]
        if gene_meta["shap_value"] > 0.2:
            text_y_new = gene_meta["shap_value"] + 0.03
            text_y_new = (
                (text_y := text_y_new)
                if text_y - text_y_new > 0.03
                else (text_y := text_y - 0.03)
            )
            plt.annotate(
                gene_meta.name,
                xy=(mod_id, gene_meta["shap_value"]),
                xytext=(mod_id + 0.2, text_y_new),
                arrowprops=dict(facecolor="black", arrowstyle="->"),
                fontsize=10,
            )

# annotate module hub genes
for mod_id, (mod, hub_gene) in enumerate(mod_hub_genes.items()):
    plt.annotate(
        hub_gene,
        xy=(mod_id, graph_nodes_df.loc[hub_gene, "shap_value"]),
        xytext=(mod_id - 0.1, -0.05),
        ha="left",
        arrowprops=dict(facecolor="black", arrowstyle="->", connectionstyle="Angle3"),
        fontweight="bold",
        fontsize=12,
    )

plt.ylabel("SHAP value", fontsize=14)
plt.xlabel("WGCNA Module", fontsize=14)
plt.xticks(
    np.arange(0, graph_nodes_df["module"].nunique(), 1),
    [f"{m}\n(n={c})" for m, c in graph_nodes_df["module"].value_counts().items()],
    fontsize=12,
)
plt.yticks(fontsize=12)
plt.title(
    "Distribution of shapely values\nWGCNA prim/norm (down-regulated DEGs)", fontsize=16
)
ax.set_axisbelow(True)
plt.grid(axis="y", zorder=-10)
plt.ylim((-0.1, 0.6))
plt.xlim((-1, graph_nodes_df["module"].nunique()))
plt.legend([], [], frameon=False)
plt.tight_layout()
plt.savefig(FIGURES_PATH.joinpath("wgcna_prim_norm_down.pdf"))
plt.savefig(FIGURES_PATH.joinpath("wgcna_prim_norm_down.svg"))

### B: hspc/prim

#### Up-regulated modules

In [ ]:
hub_genes_df = pd.read_csv(
    ROOT.joinpath("wgcna")
    .joinpath("sample_type_hspc+mcrpc+norm+prim__hspc_vs_prim_padj_0_05_up_1_0")
    .joinpath("standard")
    .joinpath("results")
    .joinpath("bicor_signed_hub_genes.csv"),
    index_col=0,
)
mod_hub_genes = hub_genes_df["SYMBOL"].to_dict()
del mod_hub_genes["M0"]
mod_hub_genes

In [14]:
network_file = (
    ROOT.joinpath("rich_network")
    .joinpath("sample_type_hspc+mcrpc+norm+prim__hspc_vs_prim_padj_0_05_up_1_0")
    .joinpath("standard")
    .joinpath("random_forest")
    .joinpath(
        "sample_type_hspc+mcrpc+norm+prim__"
        "hspc_vs_prim_padj_0_05_up_1_0_corr_th_0_1_full_wgcna_ml_net.graphml"
    )
)
graph = nx.read_graphml(network_file)

In [ ]:
graph_nodes_df = pd.DataFrame(dict(graph.nodes)).transpose().sort_values("module")
graph_nodes_df["shap_value"] = graph_nodes_df["shap_value"].astype(float)
graph_nodes_df["log2FoldChange"] = graph_nodes_df["log2FoldChange"].astype(float)
graph_nodes_df.head()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 10), dpi=300)

ax = sns.stripplot(
    data=graph_nodes_df,
    x="module",
    y="shap_value",
    hue="module",
    dodge=False,
    jitter=False,
    ax=ax,
)

ax = sns.boxplot(
    data=graph_nodes_df,
    x="module",
    y="shap_value",
    hue="module",
    dodge=False,
    ax=ax,
)

# annotate top genes by SHAP value
for mod_id, (mod, mod_df) in enumerate(graph_nodes_df.groupby("module")):
    mod_df.sort_values("shap_value", ascending=False, inplace=True)
    text_y = 99
    for i in range(min(5, len(mod_df))):
        gene_meta = mod_df.iloc[i]
        if gene_meta["shap_value"] > 0.1:
            text_y_new = gene_meta["shap_value"] + 0.03
            text_y_new = (
                (text_y := text_y_new)
                if text_y - text_y_new > 0.03
                else (text_y := text_y - 0.03)
            )
            plt.annotate(
                gene_meta.name,
                xy=(mod_id, gene_meta["shap_value"]),
                xytext=(mod_id + 0.2, text_y_new),
                arrowprops=dict(facecolor="black", arrowstyle="->"),
                fontsize=10,
            )

# annotate module hub genes
for mod_id, (mod, hub_gene) in enumerate(mod_hub_genes.items()):
    plt.annotate(
        hub_gene,
        xy=(mod_id, graph_nodes_df.loc[hub_gene, "shap_value"]),
        xytext=(mod_id - 0.1, -0.05),
        ha="left",
        arrowprops=dict(facecolor="black", arrowstyle="->", connectionstyle="Angle3"),
        fontweight="bold",
        fontsize=12,
    )

plt.ylabel("SHAP value", fontsize=14)
plt.xlabel("WGCNA Module", fontsize=14)
plt.xticks(
    np.arange(0, graph_nodes_df["module"].nunique(), 1),
    [f"{m}\n(n={c})" for m, c in graph_nodes_df["module"].value_counts().items()],
    fontsize=12,
)
plt.yticks(fontsize=12)
plt.title(
    "Distribution of shapely values\nWGCNA hspc/prim (up-regulated DEGs)", fontsize=16
)
plt.ylim((-0.1, 0.6))
plt.xlim((-1, graph_nodes_df["module"].nunique()))
ax.set_axisbelow(True)
plt.grid(axis="y", zorder=-10)
plt.legend([], [], frameon=False)
plt.tight_layout()
plt.savefig(FIGURES_PATH.joinpath("wgcna_hspc_prim_up.pdf"))
plt.savefig(FIGURES_PATH.joinpath("wgcna_hspc_prim_up.svg"))

#### Down-regulated modules

In [ ]:
hub_genes_df = pd.read_csv(
    ROOT.joinpath("wgcna")
    .joinpath("sample_type_hspc+mcrpc+norm+prim__hspc_vs_prim_padj_0_05_down_1_0")
    .joinpath("standard")
    .joinpath("results")
    .joinpath("bicor_signed_hub_genes.csv"),
    index_col=0,
)
mod_hub_genes = hub_genes_df["SYMBOL"].to_dict()
del mod_hub_genes["M0"]
mod_hub_genes

In [18]:
network_file = (
    ROOT.joinpath("rich_network")
    .joinpath("sample_type_hspc+mcrpc+norm+prim__hspc_vs_prim_padj_0_05_down_1_0")
    .joinpath("standard")
    .joinpath("random_forest")
    .joinpath(
        "sample_type_hspc+mcrpc+norm+prim__"
        "hspc_vs_prim_padj_0_05_down_1_0_corr_th_0_1_full_wgcna_ml_net.graphml"
    )
)
graph = nx.read_graphml(network_file)

In [ ]:
graph_nodes_df = pd.DataFrame(dict(graph.nodes)).transpose().sort_values("module")
graph_nodes_df["shap_value"] = graph_nodes_df["shap_value"].astype(float)
graph_nodes_df["log2FoldChange"] = graph_nodes_df["log2FoldChange"].astype(float)
graph_nodes_df.head()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 10), dpi=300)

ax = sns.stripplot(
    data=graph_nodes_df,
    x="module",
    y="shap_value",
    hue="module",
    dodge=False,
    jitter=False,
    ax=ax,
)

ax = sns.boxplot(
    data=graph_nodes_df,
    x="module",
    y="shap_value",
    hue="module",
    dodge=False,
    ax=ax,
)

# annotate top genes by SHAP value
for mod_id, (mod, mod_df) in enumerate(graph_nodes_df.groupby("module")):
    mod_df.sort_values("shap_value", ascending=False, inplace=True)
    text_y = 99
    for i in range(min(5, len(mod_df))):
        gene_meta = mod_df.iloc[i]
        if gene_meta["shap_value"] > 0.1:
            text_y_new = gene_meta["shap_value"] + 0.03
            text_y_new = (
                (text_y := text_y_new)
                if text_y - text_y_new > 0.03
                else (text_y := text_y - 0.03)
            )
            plt.annotate(
                gene_meta.name,
                xy=(mod_id, gene_meta["shap_value"]),
                xytext=(mod_id + 0.2, text_y_new),
                arrowprops=dict(facecolor="black", arrowstyle="->"),
                fontsize=10,
            )

# annotate module hub genes
for mod_id, (mod, hub_gene) in enumerate(mod_hub_genes.items()):
    try:
        plt.annotate(
            hub_gene,
            xy=(mod_id, graph_nodes_df.loc[hub_gene, "shap_value"]),
            xytext=(mod_id - 0.1, -0.05),
            ha="left",
            arrowprops=dict(
                facecolor="black", arrowstyle="->", connectionstyle="Angle3"
            ),
            fontweight="bold",
            fontsize=12,
        )
    except KeyError:
        continue

plt.ylabel("SHAP value", fontsize=14)
plt.xlabel("WGCNA Module", fontsize=14)
plt.xticks(
    np.arange(0, graph_nodes_df["module"].nunique(), 1),
    [f"{m}\n(n={c})" for m, c in graph_nodes_df["module"].value_counts().items()],
    fontsize=12,
)
plt.yticks(fontsize=12)
plt.title(
    "Distribution of shapely values\nWGCNA hspc/prim (down-regulated DEGs)", fontsize=16
)
plt.ylim((-0.1, 1.1))
plt.xlim((-1, graph_nodes_df["module"].nunique()))
ax.set_axisbelow(True)
plt.grid(axis="y", zorder=-10)
plt.legend([], [], frameon=False)
plt.tight_layout()
plt.savefig(FIGURES_PATH.joinpath("wgcna_hspc_prim_down.pdf"))
plt.savefig(FIGURES_PATH.joinpath("wgcna_hspc_prim_down.svg"))

### C: mcrpc/hspc

#### Up-regulated modules

In [ ]:
hub_genes_df = pd.read_csv(
    ROOT.joinpath("wgcna")
    .joinpath("sample_type_hspc+mcrpc+norm+prim__mcrpc_vs_hspc_padj_0_05_up_1_0")
    .joinpath("standard")
    .joinpath("results")
    .joinpath("bicor_signed_hub_genes.csv"),
    index_col=0,
)
mod_hub_genes = hub_genes_df["SYMBOL"].to_dict()
del mod_hub_genes["M0"]
mod_hub_genes

In [22]:
network_file = (
    ROOT.joinpath("rich_network")
    .joinpath("sample_type_hspc+mcrpc+norm+prim__mcrpc_vs_hspc_padj_0_05_up_1_0")
    .joinpath("standard")
    .joinpath("random_forest")
    .joinpath(
        "sample_type_hspc+mcrpc+norm+prim__"
        "mcrpc_vs_hspc_padj_0_05_up_1_0_corr_th_0_1_full_wgcna_ml_net.graphml"
    )
)
graph = nx.read_graphml(network_file)

In [ ]:
graph_nodes_df = pd.DataFrame(dict(graph.nodes)).transpose().sort_values("module")
graph_nodes_df["shap_value"] = graph_nodes_df["shap_value"].astype(float)
graph_nodes_df["log2FoldChange"] = graph_nodes_df["log2FoldChange"].astype(float)
graph_nodes_df.head()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 10), dpi=300)

ax = sns.stripplot(
    data=graph_nodes_df,
    x="module",
    y="shap_value",
    hue="module",
    dodge=False,
    jitter=False,
    ax=ax,
)

ax = sns.boxplot(
    data=graph_nodes_df,
    x="module",
    y="shap_value",
    hue="module",
    dodge=False,
    ax=ax,
)

# annotate top genes by SHAP value
for mod_id, (mod, mod_df) in enumerate(graph_nodes_df.groupby("module")):
    mod_df.sort_values("shap_value", ascending=False, inplace=True)
    text_y = 99
    for i in range(min(5, len(mod_df))):
        gene_meta = mod_df.iloc[i]
        if gene_meta["shap_value"] > 0.2:
            text_y_new = gene_meta["shap_value"] + 0.03
            text_y_new = (
                (text_y := text_y_new)
                if text_y - text_y_new > 0.03
                else (text_y := text_y - 0.03)
            )
            plt.annotate(
                gene_meta.name,
                xy=(mod_id, gene_meta["shap_value"]),
                xytext=(mod_id + 0.2, text_y_new),
                arrowprops=dict(facecolor="black", arrowstyle="->"),
                fontsize=10,
            )

# annotate module hub genes
for mod_id, (mod, hub_gene) in enumerate(mod_hub_genes.items()):
    plt.annotate(
        hub_gene,
        xy=(mod_id, graph_nodes_df.loc[hub_gene, "shap_value"]),
        xytext=(mod_id - 0.1, -0.05),
        ha="left",
        arrowprops=dict(facecolor="black", arrowstyle="->", connectionstyle="Angle3"),
        fontweight="bold",
        fontsize=12,
    )

plt.ylabel("SHAP value", fontsize=14)
plt.xlabel("WGCNA Module", fontsize=14)
plt.xticks(
    np.arange(0, graph_nodes_df["module"].nunique(), 1),
    [f"{m}\n(n={c})" for m, c in graph_nodes_df["module"].value_counts().items()],
    fontsize=12,
)
plt.yticks(fontsize=12)
plt.title(
    "Distribution of shapely values\nWGCNA mcrpc/hspc (up-regulated DEGs)", fontsize=16
)
plt.ylim((-0.1, 0.6))
plt.xlim((-1, graph_nodes_df["module"].nunique()))
ax.set_axisbelow(True)
plt.grid(axis="y", zorder=-10)
plt.legend([], [], frameon=False)
plt.tight_layout()
plt.savefig(FIGURES_PATH.joinpath("wgcna_mcrpc_hspc_up.pdf"))
plt.savefig(FIGURES_PATH.joinpath("wgcna_mcrpc_hspc_up.svg"))

#### Down-regulated modules

In [ ]:
hub_genes_df = pd.read_csv(
    ROOT.joinpath("wgcna")
    .joinpath("sample_type_hspc+mcrpc+norm+prim__mcrpc_vs_hspc_padj_0_05_down_1_0")
    .joinpath("standard")
    .joinpath("results")
    .joinpath("bicor_signed_hub_genes.csv"),
    index_col=0,
)
mod_hub_genes = hub_genes_df["SYMBOL"].to_dict()
del mod_hub_genes["M0"]
mod_hub_genes

In [26]:
network_file = (
    ROOT.joinpath("rich_network")
    .joinpath("sample_type_hspc+mcrpc+norm+prim__mcrpc_vs_hspc_padj_0_05_down_1_0")
    .joinpath("standard")
    .joinpath("random_forest")
    .joinpath(
        "sample_type_hspc+mcrpc+norm+prim__"
        "mcrpc_vs_hspc_padj_0_05_down_1_0_corr_th_0_1_full_wgcna_ml_net.graphml"
    )
)
graph = nx.read_graphml(network_file)

In [ ]:
graph_nodes_df = pd.DataFrame(dict(graph.nodes)).transpose().sort_values("module")
graph_nodes_df["shap_value"] = graph_nodes_df["shap_value"].astype(float)
graph_nodes_df["log2FoldChange"] = graph_nodes_df["log2FoldChange"].astype(float)
graph_nodes_df.head()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 10), dpi=300)

ax = sns.stripplot(
    data=graph_nodes_df,
    x="module",
    y="shap_value",
    hue="module",
    dodge=False,
    jitter=False,
    ax=ax,
)

ax = sns.boxplot(
    data=graph_nodes_df,
    x="module",
    y="shap_value",
    hue="module",
    dodge=False,
    ax=ax,
)

# annotate top genes by SHAP value
for mod_id, (mod, mod_df) in enumerate(graph_nodes_df.groupby("module")):
    mod_df.sort_values("shap_value", ascending=False, inplace=True)
    text_y = 99
    for i in range(min(5, len(mod_df))):
        gene_meta = mod_df.iloc[i]
        if gene_meta["shap_value"] > 0.2:
            text_y_new = gene_meta["shap_value"] + 0.03
            text_y_new = (
                (text_y := text_y_new)
                if text_y - text_y_new > 0.03
                else (text_y := text_y - 0.03)
            )
            plt.annotate(
                gene_meta.name,
                xy=(mod_id, gene_meta["shap_value"]),
                xytext=(mod_id + 0.2, text_y_new),
                arrowprops=dict(facecolor="black", arrowstyle="->"),
                fontsize=10,
            )

# annotate module hub genes
for mod_id, (mod, hub_gene) in enumerate(mod_hub_genes.items()):
    plt.annotate(
        hub_gene,
        xy=(mod_id, graph_nodes_df.loc[hub_gene, "shap_value"]),
        xytext=(mod_id - 0.1, -0.05),
        ha="left",
        arrowprops=dict(facecolor="black", arrowstyle="->", connectionstyle="Angle3"),
        fontweight="bold",
        fontsize=12,
    )

plt.ylabel("SHAP value", fontsize=14)
plt.xlabel("WGCNA Module", fontsize=14)
plt.xticks(
    np.arange(0, graph_nodes_df["module"].nunique(), 1),
    [f"{m}\n(n={c})" for m, c in graph_nodes_df["module"].value_counts().items()],
    fontsize=12,
)
plt.yticks(fontsize=12)
plt.title(
    "Distribution of shapely values\nWGCNA mcrpc/hspc (down-regulated DEGs)",
    fontsize=16,
)
plt.ylim((-0.1, 0.6))
plt.xlim((-1, graph_nodes_df["module"].nunique()))
ax.set_axisbelow(True)
plt.grid(axis="y", zorder=-10)
plt.legend([], [], frameon=False)
plt.tight_layout()
plt.savefig(FIGURES_PATH.joinpath("wgcna_mcrpc_hspc_down.pdf"))
plt.savefig(FIGURES_PATH.joinpath("wgcna_mcrpc_hspc_down.svg"))